# Лаб1. Оптимизационные модели. Моделирование и решение.

## Задание 1

### Постановка задачи

Мы решили открыть кофейню, которая делает два вида кофе "Раф кофе" за 400 рублей и "Капучино" за 300.

Чтобы сварить 1 чашку Раф кофе необходимо: 40 гр. зерен, 140 гр. молока и 5 гр. ванильного сахара. Для того чтобы получить одну чашку капучино нужно потратить: 30 гр. зерен, 120 гр. молока На складе есть: 500 гр. зерен, 2000 гр. молока и 40 гр. ванильного сахара.

Необходимо найти план варки кофе, обеспечивающий максимальную выручку от их реализации, учитывая, что нам не важно мнение покупателей, главное потратить весь ванильный сахар.

Требуется построить математическую модель (описать ее в блокноте при помощи Markdown), создать два блокнота  в Google Colab (среда выполнения: Python, Julia). Построить модели на соответствующем языке. Решить задачу. Подготовить один слайд для презентации результата (визуализация результата) (используйте latex и Overleaf; при создании слайда в latex примените Qwen, Deepseek или др. ).

### Математеческая модель

- $x$ — кол-во чашек кофе **раф**  
- $y$ — кол-во чашек кофе **капучино**  

Ограничения:  
- $40x + 30y \le 500$ — ограничение по зёрнам (гр.)  
- $140x + 500y \le 2000$ — ограничение по молоку (гр.)  
- $5x = 40$ — ограничение по ванильному сахару (гр.)

Ограничения целостности:
- $x \ge 0$
- $y \ge 0$
- $x \in \mathbb{Z}$
- $y \in \mathbb{Z}$

Выручка:
- $400x + 300y \to  max$

### Код

In [13]:
:dep good_lp = "1.14.0"

use good_lp::*;

let mut vars = variables!();
let x = vars.add(variable().min(0.0));
let y = vars.add(variable().min(0.0));

let problem = vars
    .maximise(400.0 * x + 300.0 * y)
    .using(default_solver)
    .with(constraint!(40.0 * x + 30.0 * y <= 500.0))
    .with(constraint!(140.0 * x + 120.0 * y <= 2000.0))
    .with(constraint!(5.0 * x == 40.0));

let solution = problem.solve()?;

println!("x = {}", solution.value(x));
println!("y = {}", solution.value(y));

let revenue = solution.eval(400.0 * x + 300.0 * y);
println!("Максимальная выручка = {}", revenue);

x = 8
y = 6
Максимальная выручка = 5000
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-5) elements
Optimal - objective value 5000
After Postsolve, objective 5000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 5000 - 0 iterations time 0.002, Presolve 0.00


## Задание 2. Задача о назначениях

### Постановка задачи (обобщённая)

Шесть рабочих W1 – W6 должны выполнить пять работ J1, J2, J3, J4, J5. Уровень подготовки и
опыт работы у рабочих разный. Время выполнения работ для каждого работника приведено в
таблице:

|       | J_1 | J_2 | J_3 | J_4 | J_5 |
|-------|-----|-----|-----|-----|-----|
|  W_1  | 3   | 4   | 7   | 4   | 8   |
|  W_2  | 4   | 8   | 8   | 3   | 3   |
|  W_3  | 7   | 5   | 7   | 2   | 7   |
|  W_4  | 5   | 7   | 9   | 5   | 9   |
|  W_5  | 4   | 6   | 5   | 3   | 8   |
|  W_6  | 7   | 5   | 5   | 4   | 7   |

Каждая работа должна быть выполнена, она выполняется без прерывания одним рабочим. Порядок выполнения работ не важен. Требуется распределить работы так, чтобы минимизировать время занятости рабочих (время, когда все рабочие выполнят запланированные для них работы). Требуется построить математическую модель и найти оптимальное решение.

### Математиеская модель

- T - момент времени, когда все рабочие завершат выполнение своей работы

$ x_{ij} = 
\begin{cases}
1 \text{ если i-й рабочий выполняет j-ю работу,} \quad i \in [1, 6], \quad j \in [1, 5], \\
0 \text{ если нет}
\end{cases}
$

### Ограничения

Каждая работа должна быть выполнена одним из рабочих:

$ \sum_{i=1}^{4} x_{ij} = 1 \quad \forall \quad j = 1,\dots,5, $

Время выполнения работы каждым из рабочих не больше \( T \):

$ (A^i, x^i) \leq T, \quad \text{при } i = 1,\dots,6, $

где в матрице $ A $ значение $ a_{ij} $ равно времени выполнения $ i $-й работы $ j $-м рабочим; $ A^i, x^i $ -- $ i $-е строки в матрицах $ A, x $.

$
A = 
\begin{bmatrix}
3 & 4 & 7 & 4 & 8 \\
4 & 8 & 8 & 3 & 3 \\
7 & 5 & 7 & 2 & 7 \\
5 & 7 & 9 & 5 & 9 \\
4 & 6 & 5 & 3 & 8 \\
7 & 5 & 5 & 4 & 7 
\end{bmatrix}
$

Критерий качества: $ T \to \min $

### Код

In [12]:
:dep good_lp = "1.14.0"

use good_lp::*;

let a = vec![
    vec![3, 4, 7, 4, 8],
    vec![4, 8, 8, 3, 3],
    vec![7, 5, 7, 2, 7],
    vec![5, 7, 9, 5, 9],
    vec![4, 6, 5, 3, 8],
    vec![7, 5, 5, 4, 7],
];

let num_workers = 6;
let num_jobs = 5;

let mut vars = variables!();

let mut x = vec![];
for i in 0..num_workers {
    let mut row = vec![];
    for j in 0..num_jobs {
        row.push(vars.add(variable().binary()));
    }
    x.push(row);
}

let t = vars.add(variable().min(0.0));

let mut problem = vars.minimise(t).using(default_solver);

for job in 0..num_jobs {
    let mut job_constraint = Expression::from_other_affine(0.0);
    for worker in 0..num_workers {
        job_constraint = job_constraint + x[worker][job];
    }
    problem.add_constraint(job_constraint.eq(1.0));
}

for worker in 0..num_workers {
    let mut time_constraint = Expression::from_other_affine(0.0);
    for job in 0..num_jobs {
        time_constraint = time_constraint + (a[worker][job] as f64) * x[worker][job];
    }
    problem.add_constraint(time_constraint.leq(t));
}

let solution = problem.solve()?;

println!("Оптимальное решение найдено!");
println!("Минимальное время выполнения: T = {} часов", solution.value(t));
println!();

println!("Распределение работ по рабочим:");
for i in 0..num_workers {
    let mut worker_time = 0.0;
    let mut jobs_assigned = Vec::new();

    for j in 0..num_jobs {
        if solution.value(x[i][j]) > 0.5 {
            worker_time += a[i][j] as f64;
            jobs_assigned.push(j + 1);
        }
    }

    if !jobs_assigned.is_empty() {
        println!("Рабочий W_{} выполняет работы: {:?}", i + 1, jobs_assigned);
        println!("  Общее время: {} часов", worker_time);
        println!("  Состав работ:");
        for &job in &jobs_assigned {
            println!("    - Работа J_{} ({} часов)", job, a[i][job - 1]);
        }
        println!();
    }
}

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Jan 21 2024 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.12632 - 0.00 seconds
Cgl0005I 5 SOS with 30 members
Cgl0004I processed model has 11 rows, 31 columns (30 integer (30 of which binary)) and 66 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 8 integers unsatisfied sum - 2.45053
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 5 iterations 11
Cbc0038I Solution found of 5
Cbc0038I Relaxing continuous gives 5
Cbc0038I Before mini branch and bound, 22 integers at bound fixed and 0 continuous
Cbc0038I Full problem 11 rows 31 columns, reduced to 8 rows 8 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I Round again with cutoff of 3.91272
Cbc0038I Pass   2: suminf.    0.91346 (7) obj. 3.91272 iterations 8
Cbc0038I Pass   3: suminf.    2.76416 (8) obj. 3.91272 iterations 8
Cbc0038I Pass   4: suminf.    2.7

()

## Задание 3

### Постановка задачи

Представьте, что мы хотим провести DataFest с 5 разными секциями и у для каждой из них мы забронировали 5 залов различной вместимости: в каждом зале не должно быть меньше 180 и больше 250 людей, а на третьей секции активность подразумевает, что должно быть точно 220 человек.

При этом организаторам удалось собрать 1000 заявок с указанием приоритета посещения 3 трех секций, где 1 - максимальный приоритет, 3 минимальный, а 10000 означает, что человек не пойдет на эту секцию.

Наша задача дать рекомендацию слушателю, на какую же секцию ему все таки пойти, чтобы хватило места всем.

### Математическая модель

- $ I = {1, 2, \dots, 1000} $ - множество людей;
- $ J = {1, 2, 3, 4, 5} $ - множество секций;
- $ P_{ij} $ - приоритет человека i для секции (1, 2, 3 или 10000);
- $ L_j = 180 $ - минимальная вместимость секций;
- $ U_j = 250 $ - максимальная вместимость секций.


$ x_{ij} = 
\begin{cases}
1 \text{ если i-й человек назначенен на секцию j,} \\
0 \text{ если нет}
\end{cases}
$

Ограниччения:

$ \sum_{i=1}^{1000} x_{ij} = 1 \quad \forall \quad j = 1,\dots,5, $

$ L_j \le \sum_{i=1}^{1000} x_{ij} \le U_j $

$ L_j \le \sum_{i=1}^{1000} x_{i 3} = 220 $

$ x \in {0, 1} $ для всех $ i \in \mathbb{I}, j \in \mathbb{J} $

### Код

In [11]:
:dep good_lp = "1.14.0"
:dep rand = "0.8"
:dep rand_distr = "0.4"

use good_lp::*;
use rand::distributions::{Distribution, WeightedIndex};
use rand::rngs::StdRng;
use rand::SeedableRng;

let n_people = 1000;
let n_sections = 5;

let min_capacity = vec![180, 180, 220, 180, 180];
let max_capacity = vec![250, 250, 220, 250, 250];

fn generate_applications(n_people: usize) -> Vec<Vec<i32>> {
    let mut rng = StdRng::seed_from_u64(123);
    let weights = [3, 3, 3, 1]; 
    
    let mut applications = Vec::new();
    
    for i in 0..n_people {
        let mut person_applications = Vec::new();
        
        for section in 0..5 {
            let dist = WeightedIndex::new(&weights).unwrap();
            let priority_value = match dist.sample(&mut rng) {
                0 => 1,
                1 => 2,
                2 => 3,
                3 => 10000,
                _ => 10000,
            };
            person_applications.push(priority_value);
        }
        
        applications.push(person_applications);
    }
    
    applications
}

let applications = generate_applications(n_people);

let mut vars = variables!();
let mut assignments: Vec<Vec<Variable>> = Vec::new();

for i in 0..n_people {
    let mut person_vars = Vec::new();
    for j in 0..n_sections {
        let var = vars.add(variable().binary().name(format!("x_{}_{}", i, j)));
        person_vars.push(var);
    }
    assignments.push(person_vars);
}

fn create_objective(assignments: &[Vec<Variable>], applications: &[Vec<i32>]) -> Expression {
    let mut objective = Expression::from(0);
    
    for i in 0..assignments.len() {
        for j in 0..assignments[i].len() {
            let cost = if applications[i][j] == 10000 {
                100000.0
            } else {
                applications[i][j] as f64
            };
            objective += cost * assignments[i][j];
        }
    }
    
    objective
}

let mut problem = vars.minimise(create_objective(&assignments, &applications))
                      .using(good_lp::default_solver);

for i in 0..n_people {
    let mut constraint_expr = Expression::from(0);
    for j in 0..n_sections {
        constraint_expr += assignments[i][j];
    }
    problem = problem.with(constraint_expr.eq(1));
}

for j in 0..n_sections {
    let mut capacity_expr = Expression::from(0);
    for i in 0..n_people {
        capacity_expr += assignments[i][j];
    }
    problem = problem.with(constraint!(capacity_expr.clone() >= min_capacity[j]));
    problem = problem.with(constraint!(capacity_expr <= max_capacity[j]));
}

let solution = problem.solve().expect("Failed to solve the problem");

let mut section_counts = vec![0; n_sections];
let mut recommendations = Vec::new();

for i in 0..n_people {
    for j in 0..n_sections {
        if solution.value(assignments[i][j]) > 0.5 {
            section_counts[j] += 1;
            recommendations.push((i, j, applications[i][j]));
            break;
        }
    }
}

println!("Результаты распределения:");
for (j, &count) in section_counts.iter().enumerate() {
    println!("Секция {}: {} человек", j + 1, count);
}

Welcome to the CBC MILP Solver 
Version: 2.10.11 
Build Date: Jan 21 2024 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 1168 - 0.01 seconds
Cgl0005I 1000 SOS with 5000 members
Cgl0004I processed model has 1010 rows, 5000 columns (5000 integer (5000 of which binary)) and 15000 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 1168
Cbc0038I Before mini branch and bound, 5000 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.16 seconds)
Cbc0038I After 0.16 seconds - Feasibility pump exiting with objective of 1168 - took 0.00 seconds
Cbc0012I Integer solution of 1168 found by feasibility pump after 0 iterations and 0 nodes (0.16 seconds)
Cbc0001I Search completed - best objective 1168, took 0 iterations and 0 nodes (0.16 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root 

()